<a href="https://colab.research.google.com/github/matjesg/deepflash2/blob/master/paper/4_experts_vs_uncertainties.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# deepflash2 - Relationship between uncertainty and expert agreement

> This notebook reproduces the results of the deepflash2 [paper](https://arxiv.org/abs/2111.06693) for the relationship between pixel-wise uncertainty and expert agreement.

- **Data and models**: Data and trained models are available on [Google Drive](https://drive.google.com/drive/folders/1r9AqP9qW9JThbMIvT0jhoA5mPxWEeIjs?usp=sharing). To use the data in Google Colab, create a [shortcut](https://support.google.com/drive/answer/9700156?hl=en&co=GENIE.Platform%3DDesktop) of the data folder in your personal Google Drive.

*Source files created with this notebook*:

`experts_vs_uncertainties.csv`

*References*:

Griebel, M., Segebarth, D., Stein, N., Schukraft, N., Tovote, P., Blum, R., & Flath, C. M. (2021). Deep-learning in the bioimaging wild: Handling ambiguous data with deepflash2. arXiv preprint arXiv:2111.06693.


## Setup

- Install dependecies
- Connect to drive

In [ ]:
!pip install deepflash2

In [ ]:
# Imports
import numpy as np
import pandas as pd
from pathlib import Path
import zarr
from deepflash2.all import *
from deepflash2.data import _read_msk

In [ ]:
# Connect to drive
from google.colab import drive
drive.mount('/gdrive')

## Settings

In [ ]:
DATASETS = ['PV_in_HC', 'cFOS_in_HC', 'mScarlet_in_PAG', 'YFP_in_CTX', 'GFAP_in_HC']
OUTPUT_PATH = Path("/content") 
DATA_PATH = Path('/gdrive/MyDrive/deepflash2-paper/data')
TRAINED_MODEL_PATH= Path('/gdrive/MyDrive/deepflash2-paper/models/')
MODEL_NO = '1'
UNCERTAINTY_BINS = np.linspace(0, 0.25, 26)

## Analysis

1. Predict segmentations and uncertainties on the test set
2. Calculate expert agreement from the expert segmentations
3. Postprocess results 

See `deepflash2_figures-and-tables.ipynb` for plots of the data.

In [ ]:
result_list = []
for dataset in DATASETS:
  test_data_path = DATA_PATH/dataset/'test'
  ensemble_path = TRAINED_MODEL_PATH/dataset/MODEL_NO
  el_pred = EnsembleLearner('images',
                            path=test_data_path, 
                            ensemble_path=ensemble_path) 
  
  # Predict and save semantic segmentation masks
  el_pred.get_ensemble_results(el_pred.files, use_tta=True)
  
  # Load expert masks      
  gt_est = GTEstimator(exp_dir='masks_experts', path=test_data_path)
  exp_averages = {} 
  for m, exps in gt_est.masks.items():
    file_id = m.split('_')[0]
    exp_masks = [_read_msk(gt_est.mask_fn(exp,m), instance_labels=gt_est.instance_labels) for exp in exps]
    exp_averages[file_id] = np.mean(exp_masks, axis=0)

  for idx, r in el_pred.df_ens.iterrows():
    file_id = r.file.split('.')[0]

    # Get prediction from softmax
    smx = zarr.load(r.softmax_path)
    pred = np.argmax(smx, axis=-1)

    # Get uncertainty maps
    unc = zarr.load(r.uncertainty_path)

    # Get expert average annotations
    exp_average = exp_averages[file_id]

    # Calculate "soft" error map
    error_map = np.abs(pred-exp_average)

    # Calculate error means (error rate)
    digitized = np.digitize(unc.flatten(), UNCERTAINTY_BINS)
    error_means = [error_map.flatten()[digitized == i].mean() for i in range(1, len(UNCERTAINTY_BINS))]

    # Calculate expert agreement
    expert_agreement = []
    for i in range(1, len(UNCERTAINTY_BINS)):
      bin_error = error_map.flatten()[digitized == i]
      expert_agreement.append((np.sum(bin_error==0) + np.sum(bin_error==1))/len(bin_error))

    df_tmp = pd.DataFrame({
      'dataset':dataset,
      'file':r.file,
      'uncertainty_bins': UNCERTAINTY_BINS[:-1],
      'error_rate': error_means,
      'expert_agreement': expert_agreement
      })
    result_list.append(df_tmp)

df = pd.concat(result_list).reset_index(drop=True)
df.to_csv(OUTPUT_PATH/'experts_vs_uncertainties.csv', index=False)